In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, r2_score
import math

In [3]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size]

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, padding):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1)
    def forward(self, x):
        return self.net(x)

class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3):
        super().__init__()
        layers = []
        for i in range(len(num_channels)):
            dilation_size = 2 ** i
            in_ch = num_inputs if i == 0 else num_channels[i-1]
            out_ch = num_channels[i]
            padding = (kernel_size - 1) * dilation_size
            layers.append(TemporalBlock(in_ch, out_ch, kernel_size, dilation=dilation_size, padding=padding))
        self.network = nn.Sequential(*layers)
        self.linear = nn.Linear(num_channels[-1], 1)
    def forward(self, x):
        y = self.network(x)
        return self.linear(y[:, :, -1])

## Data Preparation and rolling evaluation

In [5]:
# Load dataset
df = pd.read_csv("RELIANCE.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'Close']].dropna()
df = df.sort_values("Date")
df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
df['RealizedVol'] = df['LogReturn'].rolling(window=5).apply(lambda x: np.sqrt(np.mean(x**2)))
df.dropna(inplace=True)

# Normalize
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Close_scaled'] = scaler.fit_transform(df[['Close']])

# TCN inputs
def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

In [6]:
window = 20
X, y = create_sequences(df['Close_scaled'].values, window)

# Convert to torch tensors
X_tensor = torch.FloatTensor(X).unsqueeze(1)  # (N, 1, T)
y_tensor = torch.FloatTensor(y)

In [8]:
# Rolling evaluation
mae_list, rmse_list, acc_list, f1_list, vol_rmse_list = [], [], [], [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(len(X_tensor) - 30, len(X_tensor) - 1):
    model = TCN(num_inputs=1, num_channels=[16, 16])
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    X_train = X_tensor[:i].to(device)
    y_train = y_tensor[:i].to(device)

    # Train
    model.train()
    for epoch in range(50):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output.squeeze(), y_train)
        loss.backward()
        optimizer.step()

    # Predict next value
    model.eval()
    with torch.no_grad():
        pred = model(X_tensor[i].unsqueeze(0).to(device)).item()
        actual = y_tensor[i + 1].item()
        prev = y_tensor[i].item()

    # Reverse scaling
    pred_real = scaler.inverse_transform([[pred]])[0][0]
    actual_real = scaler.inverse_transform([[actual]])[0][0]
    prev_real = scaler.inverse_transform([[prev]])[0][0]

    # Price metrics
    mae_list.append(mean_absolute_error([actual_real], [pred_real]))
    rmse_list.append(math.sqrt(mean_squared_error([actual_real], [pred_real])))

    # Direction
    true_dir = int(actual_real > prev_real)
    pred_dir = int(pred_real > prev_real)
    acc_list.append(accuracy_score([true_dir], [pred_dir]))
    f1_list.append(f1_score([true_dir], [pred_dir], zero_division=1))

    # Volatility (naive)
    actual_vol = df['RealizedVol'].iloc[i + window + 1]
    pred_vol = df['RealizedVol'].iloc[i + window]
    if not np.isnan(actual_vol) and not np.isnan(pred_vol):
        vol_rmse_list.append(math.sqrt(mean_squared_error([actual_vol], [pred_vol])))

# Print results
print("\n TCN Rolling Evaluation Summary:\n")
print(f"Price Prediction: MAE = {np.mean(mae_list):.4f}, RMSE = {np.mean(rmse_list):.4f}")
print(f" Direction Prediction: Accuracy = {np.mean(acc_list):.4f}, F1 = {np.mean(f1_list):.4f}")
print(f" Volatility Prediction: RMSE = {np.mean(vol_rmse_list):.6f}")


 TCN Rolling Evaluation Summary:

Price Prediction: MAE = 70.4374, RMSE = 70.4374
 Direction Prediction: Accuracy = 0.4483, F1 = 0.4483
 Volatility Prediction: RMSE = 0.002093
